In [ ]:
!pip install pydicom

In [ ]:
!pip install np_utils

In [1]:
import pydicom
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
import math
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import h5py
import tensorflow as tf

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
#Reading Files:

data_dir = "../train"
patients = os.listdir(data_dir)
labels_df = pd.read_csv("../train_labels.csv")
label=labels_df.loc[:,'MGMT_value']
scan = ["FLAIR","T1w","T1wCE","T2w"]
labels_df.head()

,BraTS21ID,MGMT_value
0,0,1
1,2,1
2,3,0
3,5,1
4,6,1


In [3]:
i=0
for patient in patients[:10]:
#     label = labels_df._get_value(patient,"")
    path = data_dir + '/'+ patient + "/" + scan[0]
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[0]))
    print(len(slices),label[i],slices[i].pixel_array.shape)
    i+=1
#     print(slices[0])

400 1 (512, 512)
129 1 (512, 512)
129 0 (512, 512)
400 1 (512, 512)
129 1 (512, 512)
168 1 (512, 512)
240 0 (512, 512)
129 1 (512, 512)
400 1 (512, 512)
216 1 (512, 512)


In [ ]:
len(patients)

In [4]:
#FUNCTIONS WE USE:
l=[1,2,3,4,5,6,7,8,9]
def chunk(l,n):
    for i in range(0,len(l),n):
        yield l[i:i+n]

print(list(chunk(l,3)))
        
        
def mean(l):
    return sum(l)/len(l)

def resize(layer,HM_slices):
    if(len(layer)<HM_slices and HM_slices-len(layer)>1):
        layer.append(layer[-1])
        return resize(layer,HM_slices)
    elif(len(layer)<HM_slices and HM_slices-len(layer)==1):
        layer.append(layer[-1])
        return layer
    else:
        return layer
    
    
    
def process_data(patients, label_df, IMG_PX_SIZE = 50,HM_slices=20, visual=False, i=0):
    path = data_dir + '/'+ patient + "/" + scan[scan_type]
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[0]))
    i+=1

       
    new_slices = []

    slices = [cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE,IMG_PX_SIZE)) for each_slice in slices]

    chunk_sizes =  math.ceil(len(slices)/HM_slices)

    for slice_chunk in chunk(slices, chunk_sizes):
            slice_chunk = list(map(mean,zip(*slice_chunk)))
            new_slices.append(slice_chunk)

    new_slices=resize(new_slices,HM_slices)
    len(new_slices)
        
    if visual:
        fig = plt.figure()
        for num,each_slice in enumerate(new_slices):
            y = fig.add_subplot(4,5,num+1)
            y.imshow(each_slice,cmap='gray')
        #         y.imshow(each_slice)
        plt.show()
            
    if label[i]==1: label[i]=np.array([0,1])
    elif label[i]==0: label[i]=np.array([1,0])

    return np.array(new_slices), label [i]
        

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


In [6]:
IMG_PX_SIZE = 50
HM_slices=20
i=0
j=0 
much_data=[]
ps=[]
for patient in patients[100:150]:
    for scan_type in range(4):
        path = data_dir + '/'+ patient + "/" + scan[scan_type]
        slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
        slices.sort(key = lambda x: int(x.ImagePositionPatient[0]))
    #     print(len(slices),label[i],slices[i].pixel_array.shape)
        i+=1

       
        new_slices = []

        slices = [cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE,IMG_PX_SIZE)) for each_slice in slices]

        chunk_sizes =  math.ceil(len(slices)/HM_slices)
    #     print(chunk_sizes)

        for slice_chunk in chunk(slices, chunk_sizes):
    #         print(slice_chunk)
            slice_chunk = list(map(mean,zip(*slice_chunk)))
            new_slices.append(slice_chunk)

        new_slices=resize(new_slices,HM_slices)
#         print(len(new_slices))
        
#         if label[i]==1: label[i]=np.vectorize([0,1])
#         elif label[i]==0: label[i]=np.vectorize([1,0])


#         fig = plt.figure()
#         for num,each_slice in enumerate(new_slices):
#             y = fig.add_subplot(4,5,num+1)
#     #         new_image = cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE,IMG_PX_SIZE))
#             y.imshow(each_slice,cmap='gray')
#     #         y.imshow(each_slice)
        ps.append(new_slices)
    much_data.append(ps)
# plt.show()
print('I am done')
print(len(much_data),len(ps))

I am done
50 200


In [6]:
'''from scipy.ndimage import zoom
IMG_PX_SIZE = 250
i=0
j=0 
for patient in patients[:1]:
    path = data_dir + '/'+ patient + "/" + scan[0]
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[0]))
    i+=1
    print(len(slices))
    HM_slices=20
#     new_slices = []
    factor = len(slices)/20
    new_slices = zoom(np.array(slices),(1,1,factor))
    print(type(np.array(slices)))
    print(np.array(slices).shape)
      
#     for slice_chunk in chunk(slices, chunk_sizes):
#         slice_chunk = list(map(mean,zip(*slice_chunk)))
#         new_slices.append(slice_chunk)'''

'from scipy.ndimage import zoom\nIMG_PX_SIZE = 250\ni=0\nj=0 \nfor patient in patients[:1]:\n    path = data_dir + \'/\'+ patient + "/" + scan[0]\n    slices = [pydicom.read_file(path + \'/\' + s) for s in os.listdir(path)]\n    slices.sort(key = lambda x: int(x.ImagePositionPatient[0]))\n    i+=1\n    print(len(slices))\n    HM_slices=20\n#     new_slices = []\n    factor = len(slices)/20\n    new_slices = zoom(np.array(slices),(1,1,factor))\n    print(type(np.array(slices)))\n    print(np.array(slices).shape)\n      \n#     for slice_chunk in chunk(slices, chunk_sizes):\n#         slice_chunk = list(map(mean,zip(*slice_chunk)))\n#         new_slices.append(slice_chunk)'

In [ ]:
sample_shape = (16, 16, 16, 3)
#MODEL
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=sample_shape))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()
# Fit data to model
history = model.fit(much_data, label[100:150],
            batch_size=128,
            epochs=40,
            verbose=1,
            validation_split=0.3)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 14, 14, 14, 32)    2624      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 7, 7, 7, 32)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 7, 7, 7, 32)       128       
_________________________________________________________________
dropout (Dropout)            (None, 7, 7, 7, 32)       0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 5, 5, 5, 64)       55360     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 2, 2, 2, 64)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 2, 2, 2, 64)       2

In [8]:
# len(new_slices)
label


0      1
1      1
2      0
3      1
4      1
      ..
580    1
581    1
582    1
583    0
584    0
Name: MGMT_value, Length: 585, dtype: int64